In [2]:
import torchaudio

waveform, sr = torchaudio.load("../data/6282e2e2479eff12a562838c_q4_personality.wav")
waveform.size()

torch.Size([2, 8274042])

In [4]:
torchaudio.functional.vad(waveform, sr).size()

torch.Size([2, 8256402])

In [7]:
import torch
from transformers import Wav2Vec2ForXVector, Wav2Vec2Processor

model_id = "facebook/wav2vec2-base"
device = torch.device("cuda")
model = Wav2Vec2ForXVector.from_pretrained(model_id, weights_only=True) # use_safetensors=True, 
model = model.half().to(device)
model.eval()
processor = Wav2Vec2Processor.from_pretrained(model_id)

signal = processor(  # noqa: F722 # type: ignore
    waveform, sampling_rate=16000, return_tensors="pt"
).input_values.squeeze(0)
signal = signal.half().to(device)

chunks = torch.split(signal, 120 * 16000, dim=1)  # 15-second chunks
embeddings = []
with torch.no_grad():
    for chunk in chunks:
        embeddings.append(model(chunk).embeddings)

audio_features = torch.mean(torch.stack(embeddings), dim=0)

/home/solan/repos/airi-research/.venv/lib/python3.8/site-packages/transformers/configuration_utils.py:306: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of Wav2Vec2ForXVector were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'feature_extractor.bias', 'feature_extractor.weight', 'objective.weight', 'projector.bias', 'projector.weight', 'tdnn.0.kernel.bias', 'tdnn.0.kernel.weight', 'tdnn.1.kernel.bias', 'tdnn.1.kernel.weight', 'tdnn.2.kernel.bias', 'tdnn.2.kernel.weight', 'tdnn.3.kernel.bias', 'tdnn.3.kernel.weight', 'tdnn.4.kernel.bias', 'tdnn.4.kernel.weight']
You should probably TRAIN this model on a down-stream task to be able to 

In [9]:
audio_features.size()

torch.Size([2, 512])